# Blűcher

## Introduction


In [49]:
from functools import reduce
from math import comb, floor

def choose(n, k):
    '''math.comb with the convention that k < 0 is 0'''
    if k < 0:
        return 0
    else:
        return comb(n, k)

## Fire

Fire combat is given by a number of dice (for an artillery unit, the current ammunition level of the unit; for an infantry unit, the current elan of the unit), possibly with a penalty or a bonus. Normally, every 6 rolled gives one hit; with the bonus *one* five rolled gives an additional hit. The number of dice may be halved due to penalties to firing.

For infantry and cavalry, each hit causes one fatigue. For artillery, 1 hit causes a retreat; more than one hit (during a single fire phase) causes the artillery unit to **retire.**

See [Probability of i event_A and j event_B on n dice](https://boardgames.stackexchange.com/questions/54117/probability-of-i-event-a-and-j-event-b-on-n-dice) for the computations below.

In [50]:
def fireRoll(n, bonus=False):
    result = [0] * (n + 1)
    for i in range(0, n + 1):
        # (number of ways of choosing i dice * succ^i * fail^(dice - i))
        if not bonus:
            result[i] = (
                choose(n, i) 
                * pow(1, i)
                * pow(5, n - i)
            )
        else:
            pSixes    =  (
                choose(n, i)
                * pow(1, i)
                * pow(4, n - i)
            )
            pBonus    =  (
                choose(n, i-1)
                * pow(1, i-1)
                * (pow(5, n - (i-1)) - pow(4, n - (i-1)))
            )
            result[i] = (pSixes + pBonus)
        # finally, divide by 6^n
        result[i] = result[i] / pow(6, n)
    return result

def fire(n, bonus=False):
    exp = reduce(
        lambda acc, e: (acc + (e[0] * e[1])),
        enumerate(fireRoll(n, bonus)),
        0
    )
    return f'{floor((exp * 1000) + 0.5) / 1000}'

### Artillery

This table gives the number of fire hits, based on the number of dice rolled.

| Ammo (dice) | No bonus | bonus |
|:-----------:|:---------|:------|
| 6           | {{ fire(6) }} | {{ fire(6, True) }} |
| 5           | {{ fire(5) }} | {{ fire(5, True) }} |
| 4           | {{ fire(4) }} | {{ fire(4, True) }} |
| 3           | {{ fire(3) }} | {{ fire(3, True) }} |
| 2           | {{ fire(2) }} | {{ fire(2, True) }} |
| 1           | {{ fire(1) }} | {{ fire(1, True) }} |


### Infantry

This table gives the number of fire hits, based on the number of dice rolled.

| Elan (dice) | No bonus | bonus |
|:-----------:|:---------|:------|
| 8           | {{ fire(8) }} | {{ fire(8, True) }} |
| 7           | {{ fire(7) }} | {{ fire(7, True) }} |
| 6           | {{ fire(6) }} | {{ fire(6, True) }} |
| 5           | {{ fire(5) }} | {{ fire(5, True) }} |
| 4           | {{ fire(4) }} | {{ fire(4, True) }} |
| 3           | {{ fire(3) }} | {{ fire(3, True) }} |
| 2           | {{ fire(2) }} | {{ fire(2, True) }} |
| 1           | {{ fire(1) }} | {{ fire(1, True) }} |

For infantry and cavalry, each hit causes one fatigue. For artillery, 1 hit causes a retreat; more than one hit (during a single fire phase) causes the artillery unit to **retire.**

# Combat

Combat in Blucher, as opposed to shooting, involves the attacker and defender each rolling some number of dice and counting successes, with the one with more successes being the winner. The defender wins ties. The number of dice is based on an attribute of the unit, either its elan or its current ammunition, for artillery, with modifiers adding and removing dice.

In [51]:
def combatRoll(n, minSuccess = 4, rerolling = False):
    nSuccesses = 6 - minSuccess + 1
    result = [0.0] * (n + 1)
    for i in range(0, n + 1):
        result[i] = (
            choose(n, i)
            * pow(nSuccesses, i)
            * pow(6 - nSuccesses, n - i)
        ) / pow(6, n)
    if rerolling:
        reroll = [0.0] * (n + 1)
        reroll[0] = result[0]
        for i in range(1, n + 1):
            newRoll = combatRoll(i, minSuccess)
            for j in range(0, len(newRoll)):
                reroll[j] += result[i] * newRoll[j]
        result = reroll
    return result

def combat(n, minSuccess = 4, rerolling = False):
    exp = reduce(
        lambda acc, e: (acc + (e[0] * e[1])),
        enumerate(combatRoll(n, minSuccess, rerolling)),
        0
    )
    return f'{floor((exp * 1000) + 0.5) / 1000}'

Under some circumstances, primarily having to do with cavalry attacking or defending against infantry, either the attacker or defender's successes have to be rerolled.

The following table shows the expected results of rolling dice with and without rerolling successes.

| Dice | Without rerolling | Rerolling hits |
|:----:|:------------------|:---------------|
| 9    | {{ combat(9) }}   | {{ combat(9, rerolling=True) }} |
| 8    | {{ combat(8) }}   | {{ combat(8, rerolling=True) }} |
| 7    | {{ combat(7) }}   | {{ combat(7, rerolling=True) }} |
| 6    | {{ combat(6) }}   | {{ combat(6, rerolling=True) }} |
| 5    | {{ combat(5) }}   | {{ combat(5, rerolling=True) }} |
| 4    | {{ combat(4) }}   | {{ combat(4, rerolling=True) }} |
| 3    | {{ combat(3) }}   | {{ combat(3, rerolling=True) }} |
| 2    | {{ combat(2) }}   | {{ combat(2, rerolling=True) }} |
| 1    | {{ combat(1) }}   | {{ combat(1, rerolling=True) }} |


In [52]:
def combatResult(atk, dfs, minSuccess=4, atkReroll=False, dfsReroll=False):
    atkRoll = combatRoll(atk, rerolling=atkReroll)
    dfsRoll = combatRoll(dfs, rerolling=dfsReroll)
    atkWins = 0.0
    dfsWins = 0.0
    for a in range(0, len(atkRoll)):
        for d in range(0, len(dfsRoll)):
            if a <= d:
                dfsWins += atkRoll[a] * dfsRoll[d]
            else:
                atkWins += atkRoll[a] * dfsRoll[d]
    return [atkWins, dfsWins]

def r(atk, dfs, atkReroll=False, dfsReroll=False):
    def fmt(n): return floor((n * 100) + 0.5)
    res = combatResult(atk, dfs, atkReroll=atkReroll, dfsReroll=dfsReroll)
    res = list(map(fmt, res))
    return f'{res[0]}% / {res[1]}%'

def s(atk, dfs): return r(atk, dfs, atkReroll=True)
def t(atk, dfs): return r(atk, dfs, dfsReroll=True)

The next table shows the probability of attacker/defender winning based on the number of dice rolled. Attacker dice are along the left, defender dice are along the top. This table assumes neither rerolls successes.

|ATK\\DFS| 2 | 3 | 4 | 5 |
|:--------:|:-:|:-:|:-:|:-:|
| 2 | {{ r(2,2) }} | {{ r(2,3) }} | {{ r(2,4) }} | {{ r(2,5) }} |
| 3 | {{ r(3,2) }} | {{ r(3,3) }} | {{ r(3,4) }} | {{ r(3,5) }} |
| 4 | {{ r(4,2) }} | {{ r(4,3) }} | {{ r(4,4) }} | {{ r(4,5) }} |
| 5 | {{ r(5,2) }} | {{ r(5,3) }} | {{ r(5,4) }} | {{ r(5,5) }} |
| 6 | {{ r(6,2) }} | {{ r(6,3) }} | {{ r(6,4) }} | {{ r(6,5) }} |
| 7 | {{ r(7,2) }} | {{ r(7,3) }} | {{ r(7,4) }} | {{ r(7,5) }} |
| 8 | {{ r(8,2) }} | {{ r(8,3) }} | {{ r(8,4) }} | {{ r(8,5) }} |
| 9 | {{ r(9,2) }} | {{ r(9,3) }} | {{ r(9,4) }} | {{ r(9,5) }} |
| 10 | {{ r(10,2) }} | {{ r(10,3) }} | {{ r(10,4) }} | {{ r(10,5) }} |

|ATK\\DFS| 6 | 7 | 8 | 9 | 10 |
|:--------:|:-:|:-:|:-:|:-:|:-:|
| 2 | {{ r(2,6) }} | {{ r(2,7) }} | {{ r(2,8) }} | {{ r(2,9) }} | {{ r(2,10) }} |
| 3 | {{ r(3,6) }} | {{ r(3,7) }} | {{ r(3,8) }} | {{ r(3,9) }} | {{ r(3,10) }} |
| 4 | {{ r(4,6) }} | {{ r(4,7) }} | {{ r(4,8) }} | {{ r(4,9) }} | {{ r(4,10) }} |
| 5 | {{ r(5,6) }} | {{ r(5,7) }} | {{ r(5,8) }} | {{ r(5,9) }} | {{ r(5,10) }} |
| 6 | {{ r(6,6) }} | {{ r(6,7) }} | {{ r(6,8) }} | {{ r(6,9) }} | {{ r(6,10) }} |
| 7 | {{ r(7,6) }} | {{ r(7,7) }} | {{ r(7,8) }} | {{ r(7,9) }} | {{ r(7,10) }} |
| 8 | {{ r(8,6) }} | {{ r(8,7) }} | {{ r(8,8) }} | {{ r(8,9) }} | {{ r(8,10) }} |
| 9 | {{ r(9,6) }} | {{ r(9,7) }} | {{ r(9,8) }} | {{ r(9,9) }} | {{ r(9,10) }} |
| 10 | {{ r(10,6) }} | {{ r(10,7) }} | {{ r(10,8) }} | {{ r(10,9) }} | {{ r(10,10) }} |

In general for cavalry and infantry, when the attacker wins, the attacker takes a fatigue and may advance while the defender takes the difference as fatigues and must retreat if it is not broken. When the defender wins, the attacker takes two fatigues and must retreat while the defender takes one fatigue.

When cavalry attacks infantry, or vice-versa, a number of different rules come into use. If the infantry is *prepared* (think of this as in a square formation), the attacking cavalry must reroll its successes. If the infantry is attacking, it cannot be prepared and must reroll successes.

This table assumes the attacker rerolls successes.

|ATK\\DFS| 2 | 3 | 4 | 5 |
|:--------:|:-:|:-:|:-:|:-:|
| 2 | {{ s(2,2) }} | {{ s(2,3) }} | {{ s(2,4) }} | {{ s(2,5) }} |
| 3 | {{ s(3,2) }} | {{ s(3,3) }} | {{ s(3,4) }} | {{ s(3,5) }} |
| 4 | {{ s(4,2) }} | {{ s(4,3) }} | {{ s(4,4) }} | {{ s(4,5) }} |
| 5 | {{ s(5,2) }} | {{ s(5,3) }} | {{ s(5,4) }} | {{ s(5,5) }} |
| 6 | {{ s(6,2) }} | {{ s(6,3) }} | {{ s(6,4) }} | {{ s(6,5) }} |
| 7 | {{ s(7,2) }} | {{ s(7,3) }} | {{ s(7,4) }} | {{ s(7,5) }} |
| 8 | {{ s(8,2) }} | {{ s(8,3) }} | {{ s(8,4) }} | {{ s(8,5) }} |
| 9 | {{ s(9,2) }} | {{ s(9,3) }} | {{ s(9,4) }} | {{ s(9,5) }} |
| 10 | {{ s(10,2) }} | {{ s(10,3) }} | {{ s(10,4) }} | {{ s(10,5) }} |

|ATK\\DFS| 6 | 7 | 8 | 9 | 10 |
|:--------:|:-:|:-:|:-:|:-:|:-:|
| 2 | {{ s(2,6) }} | {{ s(2,7) }} | {{ s(2,8) }} | {{ s(2,9) }} | {{ s(2,10) }} |
| 3 | {{ s(3,6) }} | {{ s(3,7) }} | {{ s(3,8) }} | {{ s(3,9) }} | {{ s(3,10) }} |
| 4 | {{ s(4,6) }} | {{ s(4,7) }} | {{ s(4,8) }} | {{ s(4,9) }} | {{ s(4,10) }} |
| 5 | {{ s(5,6) }} | {{ s(5,7) }} | {{ s(5,8) }} | {{ s(5,9) }} | {{ s(5,10) }} |
| 6 | {{ s(6,6) }} | {{ s(6,7) }} | {{ s(6,8) }} | {{ s(6,9) }} | {{ s(6,10) }} |
| 7 | {{ s(7,6) }} | {{ s(7,7) }} | {{ s(7,8) }} | {{ s(7,9) }} | {{ s(7,10) }} |
| 8 | {{ s(8,6) }} | {{ s(8,7) }} | {{ s(8,8) }} | {{ s(8,9) }} | {{ s(8,10) }} |
| 9 | {{ s(9,6) }} | {{ s(9,7) }} | {{ s(9,8) }} | {{ s(9,9) }} | {{ s(9,10) }} |
| 10 | {{ s(10,6) }} | {{ s(10,7) }} | {{ s(10,8) }} | {{ s(10,9) }} | {{ s(10,10) }} |


On the other hand, cavalry attacking unprepared infantry forces the infantry to reroll successes.

This table assumes the defender rerolls successes.

|ATK\\DFS| 2 | 3 | 4 | 5 |
|:--------:|:-:|:-:|:-:|:-:|
| 2 | {{ t(2,2) }} | {{ t(2,3) }} | {{ t(2,4) }} | {{ t(2,5) }} |
| 3 | {{ t(3,2) }} | {{ t(3,3) }} | {{ t(3,4) }} | {{ t(3,5) }} |
| 4 | {{ t(4,2) }} | {{ t(4,3) }} | {{ t(4,4) }} | {{ t(4,5) }} |
| 5 | {{ t(5,2) }} | {{ t(5,3) }} | {{ t(5,4) }} | {{ t(5,5) }} |
| 6 | {{ t(6,2) }} | {{ t(6,3) }} | {{ t(6,4) }} | {{ t(6,5) }} |
| 7 | {{ t(7,2) }} | {{ t(7,3) }} | {{ t(7,4) }} | {{ t(7,5) }} |
| 8 | {{ t(8,2) }} | {{ t(8,3) }} | {{ t(8,4) }} | {{ t(8,5) }} |
| 9 | {{ t(9,2) }} | {{ t(9,3) }} | {{ t(9,4) }} | {{ t(9,5) }} |
| 10 | {{ t(10,2) }} | {{ t(10,3) }} | {{ t(10,4) }} | {{ t(10,5) }} |

|ATK\\DFS| 6 | 7 | 8 | 9 | 10 |
|:--------:|:-:|:-:|:-:|:-:|:-:|
| 2 | {{ t(2,6) }} | {{ t(2,7) }} | {{ t(2,8) }} | {{ t(2,9) }} | {{ t(2,10) }} |
| 3 | {{ t(3,6) }} | {{ t(3,7) }} | {{ t(3,8) }} | {{ t(3,9) }} | {{ t(3,10) }} |
| 4 | {{ t(4,6) }} | {{ t(4,7) }} | {{ t(4,8) }} | {{ t(4,9) }} | {{ t(4,10) }} |
| 5 | {{ t(5,6) }} | {{ t(5,7) }} | {{ t(5,8) }} | {{ t(5,9) }} | {{ t(5,10) }} |
| 6 | {{ t(6,6) }} | {{ t(6,7) }} | {{ t(6,8) }} | {{ t(6,9) }} | {{ t(6,10) }} |
| 7 | {{ t(7,6) }} | {{ t(7,7) }} | {{ t(7,8) }} | {{ t(7,9) }} | {{ t(7,10) }} |
| 8 | {{ t(8,6) }} | {{ t(8,7) }} | {{ t(8,8) }} | {{ t(8,9) }} | {{ t(8,10) }} |
| 9 | {{ t(9,6) }} | {{ t(9,7) }} | {{ t(9,8) }} | {{ t(9,9) }} | {{ t(9,10) }} |
| 10 | {{ t(10,6) }} | {{ t(10,7) }} | {{ t(10,8) }} | {{ t(10,9) }} | {{ t(10,10) }} |

### Artillery defending

When an artillery unit is attacked by a cavalry or infantry unit, if the cavalry or infantry gets twice as many, or more, successes than the artillery, the artillery is broken.

In [53]:
def artilleryResult(atk, dfs, minSuccess=4):
    atkRoll = combatRoll(atk)
    dfsRoll = combatRoll(dfs)
    dfsWins = 0.0
    atkWins = 0.0
    atkDbl  = 0.0
    for a in range(0, len(atkRoll)):
        for d in range(0, len(dfsRoll)):
            if a <= d:
                dfsWins += atkRoll[a] * dfsRoll[d]
            elif a < 2*d:
                atkWins += atkRoll[a] * dfsRoll[d]
            else:
                atkDbl += atkRoll[a] * dfsRoll[d]
    return [atkWins, atkDbl, dfsWins]

def a(atk, dfs):
    def fmt(n): return floor((n * 100) + 0.5)
    result = list(map(fmt, artilleryResult(atk, dfs)))
    return f'{result[1]}% / {result[0]}%'

The next chart presents the chances of the artillery breaking and the chances of the artillery merely being forced to retreat based on the number of dice rolled. (The other option is the defending artillery winning.)


|ATK\\DFS| 2 | 3 | 4 | 5 |
|:--------:|:-:|:-:|:-:|:-:|
| 2 | {{ a(2,2) }} | {{ a(2,3) }} | {{ a(2,4) }} | {{ a(2,5) }} |
| 3 | {{ a(3,2) }} | {{ a(3,3) }} | {{ a(3,4) }} | {{ a(3,5) }} |
| 4 | {{ a(4,2) }} | {{ a(4,3) }} | {{ a(4,4) }} | {{ a(4,5) }} |
| 5 | {{ a(5,2) }} | {{ a(5,3) }} | {{ a(5,4) }} | {{ a(5,5) }} |
| 6 | {{ a(6,2) }} | {{ a(6,3) }} | {{ a(6,4) }} | {{ a(6,5) }} |
| 7 | {{ a(7,2) }} | {{ a(7,3) }} | {{ a(7,4) }} | {{ a(7,5) }} |
| 8 | {{ a(8,2) }} | {{ a(8,3) }} | {{ a(8,4) }} | {{ a(8,5) }} |
| 9 | {{ a(9,2) }} | {{ a(9,3) }} | {{ a(9,4) }} | {{ a(9,5) }} |
| 10 | {{ a(10,2) }} | {{ a(10,3) }} | {{ a(10,4) }} | {{ a(10,5) }} |

|ATK\\DFS| 6 | 7 | 8 | 9 | 10 |
|:--------:|:-:|:-:|:-:|:-:|:-:|
| 2 | {{ a(2,6) }} | {{ a(2,7) }} | {{ a(2,8) }} | {{ a(2,9) }} | {{ a(2,10) }} |
| 3 | {{ a(3,6) }} | {{ a(3,7) }} | {{ a(3,8) }} | {{ a(3,9) }} | {{ a(3,10) }} |
| 4 | {{ a(4,6) }} | {{ a(4,7) }} | {{ a(4,8) }} | {{ a(4,9) }} | {{ a(4,10) }} |
| 5 | {{ a(5,6) }} | {{ a(5,7) }} | {{ a(5,8) }} | {{ a(5,9) }} | {{ a(5,10) }} |
| 6 | {{ a(6,6) }} | {{ a(6,7) }} | {{ a(6,8) }} | {{ a(6,9) }} | {{ a(6,10) }} |
| 7 | {{ a(7,6) }} | {{ a(7,7) }} | {{ a(7,8) }} | {{ a(7,9) }} | {{ a(7,10) }} |
| 8 | {{ a(8,6) }} | {{ a(8,7) }} | {{ a(8,8) }} | {{ a(8,9) }} | {{ a(8,10) }} |
| 9 | {{ a(9,6) }} | {{ a(9,7) }} | {{ a(9,8) }} | {{ a(9,9) }} | {{ a(9,10) }} |
| 10 | {{ a(10,6) }} | {{ a(10,7) }} | {{ a(10,8) }} | {{ a(10,9) }} | {{ a(10,10) }} |
